## knock 036 トランザクションとマスタを内部結合する

In [9]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列が省略されないようにする

polars.config.Config

### データを読み込む

In [10]:
df_receipt = pl.read_csv("../docker/work/data/receipt.csv")
display(df_receipt.head())

df_store = pl.read_csv("../docker/work/data/store.csv")
display(df_store.head())

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,i64,str,str,str,str,f64,f64,f64
"""S12014""","""千草台店""",12,"""千葉県""","""千葉県千葉市稲毛区千草台一丁…","""チバケンチバシイナゲクチグサ…","""043-123-4003""",140.118,35.63559,1698.0
"""S13002""","""国分寺店""",13,"""東京都""","""東京都国分寺市本多二丁目""","""トウキョウトコクブンジシホン…","""042-123-4008""",139.4802,35.70566,1735.0
"""S14010""","""菊名店""",14,"""神奈川県""","""神奈川県横浜市港北区菊名一丁…","""カナガワケンヨコハマシコウホ…","""045-123-4032""",139.6326,35.50049,1732.0
"""S14033""","""阿久和店""",14,"""神奈川県""","""神奈川県横浜市瀬谷区阿久和西…","""カナガワケンヨコハマシセヤク…","""045-123-4043""",139.4961,35.45918,1495.0
"""S14036""","""相模原中央店""",14,"""神奈川県""","""神奈川県相模原市中央二丁目""","""カナガワケンサガミハラシチュ…","""042-123-4045""",139.3716,35.57327,1679.0


### ノック

In [13]:
(
    df_receipt
    .join(other = df_store[["store_cd", "store_name"]],
          on = "store_cd",
          how = "inner")
    .head(n = 10)
)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
i64,i64,str,i64,i64,str,str,i64,i64,str
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158,"""葛が谷店"""
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81,"""成城店"""
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170,"""二ツ橋店"""
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25,"""新山下店"""
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90,"""大和店"""
20190605,1559692800,"""S13003""",1112,1,"""CS003515000195…","""P050102002""",1,138,"""狛江店"""
20181205,1543968000,"""S14024""",1102,2,"""CS024514000042…","""P080101005""",1,30,"""三田店"""
20190922,1569110400,"""S14040""",1102,1,"""CS040415000178…","""P070501004""",1,128,"""長津田店"""
20170504,1493856000,"""S13020""",1112,2,"""ZZ000000000000…","""P071302010""",1,770,"""十条仲原店"""
